In [1]:
# import all the required libraries
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
warnings.filterwarnings("ignore")

In [3]:
#read file into python dataframes
la_frame=pd.read_excel('LA 2012.xls')

### la_frame Cleaning

In [4]:
print(la_frame.shape)
la_frame.info()

(11516, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11516 entries, 0 to 11515
Data columns (total 15 columns):
ST_CODE                  11516 non-null object
ST_NAME                  11516 non-null object
MONTH                    11516 non-null int64
YEAR                     11516 non-null int64
DIST_NAME                11516 non-null object
AC_NO                    11516 non-null int64
AC_NAME                  11516 non-null object
AC_TYPE                  11516 non-null object
CAND_NAME                11516 non-null object
CAND_SEX                 11516 non-null object
CAND_CATEGORY            11516 non-null object
CAND_AGE                 11516 non-null int64
PARTYABBRE               11516 non-null object
TOTALVALIDVOTESPOLLED    11516 non-null int64
POSITION                 11516 non-null int64
dtypes: int64(6), object(9)
memory usage: 1.3+ MB


In [5]:
# Select only the utterpradesh data
la_frame.ST_NAME.unique()

array(['Goa', 'Manipur', 'Punjab', 'Uttar Pradesh', 'Uttarakhand',
       'Gujarat', 'Himachal Pradesh'], dtype=object)

In [6]:
la_frame_up=la_frame[la_frame['ST_NAME']=='Uttar Pradesh']

In [7]:
print(la_frame_up.shape)
la_frame_up.info()

(7031, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7031 entries, 1572 to 8602
Data columns (total 15 columns):
ST_CODE                  7031 non-null object
ST_NAME                  7031 non-null object
MONTH                    7031 non-null int64
YEAR                     7031 non-null int64
DIST_NAME                7031 non-null object
AC_NO                    7031 non-null int64
AC_NAME                  7031 non-null object
AC_TYPE                  7031 non-null object
CAND_NAME                7031 non-null object
CAND_SEX                 7031 non-null object
CAND_CATEGORY            7031 non-null object
CAND_AGE                 7031 non-null int64
PARTYABBRE               7031 non-null object
TOTALVALIDVOTESPOLLED    7031 non-null int64
POSITION                 7031 non-null int64
dtypes: int64(6), object(9)
memory usage: 878.9+ KB


In [8]:
# Delete unnecessary columns 
la_frame_up.drop(columns=['ST_CODE','ST_NAME','MONTH','YEAR','AC_NO'],axis=1,inplace=True)

In [9]:
# Convert all the charecters in all the columns to small case. Remove any spaces in between.Remove trailing and leading spaces.
la_frame_up.DIST_NAME=la_frame_up.DIST_NAME.str.lower().str.strip().str.replace(' ','')
la_frame_up.AC_NAME=la_frame.AC_NAME.str.lower().str.strip().str.replace(' ','')
la_frame_up.CAND_NAME=la_frame_up.CAND_NAME.str.lower().str.strip().str.replace(' ','')
la_frame.PARTYABBRE=la_frame.PARTYABBRE.str.lower().str.strip().str.replace(' ','')

#### DIST_NAME Cleaning

In [10]:
# Check DIST_NAME
print(la_frame_up.DIST_NAME.nunique())
a=list(np.sort(la_frame_up.DIST_NAME.unique()))

75


In [11]:
# Download data from www.data.gov.in which has the state wise district and town details as per census 2011 and read file into python.
dist_data=pd.read_excel('udir_up.xls')

In [12]:
dist_data.head(2)

,ST Code,STATE NAME,DT Code,DISTRICT NAME,SDT Code,SUB-DISTRICT NAME,Town Code,Area Name
0,9,UTTAR PRADESH,0,UTTAR PRADESH,0,UTTAR PRADESH,0,UTTAR PRADESH
1,9,UTTAR PRADESH,132,Saharanpur,0,Saharanpur,0,Saharanpur


In [13]:
dist_data['DISTRICT NAME']=dist_data['DISTRICT NAME'].str.lower().str.strip().str.replace(' ','')

In [14]:
b=np.sort(dist_data['DISTRICT NAME'].unique())

In [15]:
print(a==b)

False


In [16]:
print(a)
print(b)

['agra', 'aligarh', 'allahabad', 'ambedkarnagar', 'auraiya', 'azamgarh', 'baghpat', 'baharaich', 'ballia', 'balrampur', 'banda', 'barabanki', 'bareilly', 'basti', 'bheemnagar', 'bijnor', 'budaun', 'bulandsahar', 'chandauli', 'chhatrapatishahujimaharajna', 'chitrakoot', 'deoria', 'etah', 'etawah', 'faizabad', 'farrukhabad', 'fatehpur', 'firozabad', 'gautambuddhanagar', 'ghaziabad', 'ghazipur', 'gonda', 'gorakhpur', 'hamirpur', 'hardoi', 'jalaun', 'jaunpur', 'jhansi', 'jyotibaphulenagar', 'kannauj', 'kanpurnagar', 'kanshiramnagar', 'kaushambi', 'kheri', 'kushinagar', 'lalitpur', 'lucknow', 'mahamayanagar', 'maharajganj', 'mahoba', 'manpuri', 'mathura', 'mau', 'meerut', 'mirzapur', 'moradabad', 'muzaffarnagar', 'panchsheelnagar', 'pilibhit', 'prabuddhanagar', 'pratapgarh', 'raebareli', 'ramabainagar', 'rampur', 'saharanpur', 'santkabirnagar', 'santravidasnagar', 'shahjahanpur', 'shrawasti', 'siddharthnagar', 'sitapur', 'sonbhadra', 'sultanpur', 'unnao', 'varanasi']
['agra' 'aligarh' 'alla

#### Roughly matches the 2011 Census. More recent census is not avilable so, the changes done recently are not avilable in b. 

#### AC_NAME cleaning

In [17]:
la_frame_up.AC_NAME.nunique()

403

#### AC_TYPE

In [18]:
print(la_frame_up.AC_TYPE.nunique())
print(la_frame_up.AC_TYPE.unique())

2
['GEN' 'SC']


In [19]:
print(la_frame_up.CAND_NAME.nunique())
la_frame_up.CAND_NAME.value_counts()

5318


ashokkumar                                      26
omprakash                                       23
rajkumar                                        21
anilkumar                                       21
sanjay                                          19
rajeshkumar                                     19
vinodkumar                                      17
manojkumar                                      17
rajesh                                          16
rajaram                                         16
vijaykumar                                      16
shivkumar                                       16
santoshkumar                                    15
suresh                                          15
sunilkumar                                      14
arunkumar                                       13
arvind                                          12
ajaykumar                                       12
dineshkumar                                     11
ramesh                         

In [20]:
la_frame_up=la_frame_up[la_frame_up['CAND_NAME']!='noneoftheabove']


In [21]:
# Check how many NaNs are present in all the columns
la_frame_up.shape

(7031, 10)

In [22]:
print(la_frame_up.PARTYABBRE.nunique())
la_frame_up.PARTYABBRE.value_counts().sort_values()

223


BMSKM            1
BJS              1
BCB              1
ABAS             1
BJVP             1
PRBD             1
KiRP             1
JND              1
LSWP             1
APVM             1
BhSaPa           1
MNDP             1
DBJ              1
AP               1
NJC              1
BRM              1
RBHP             1
BhJaSaP          1
RSUPRP           1
JKiP             1
SPYSCB           1
BRKD             1
RaAD             1
BhRapa           1
PSS              1
AHNP             1
JaVM             1
RDHP             1
BJBCD            1
GSPS             1
ABLTC            1
DSP              1
MSP              1
ABRS             1
NDPF             1
NRSN             1
PrDP             1
PAP              1
SaPP             1
GMM              1
RSVC             1
SWPI             1
BhJD             1
BMKCP            1
AMP              1
JSMP             1
AMKP             1
RJAP             1
ABLTP            1
BHSVP            1
LSP              1
SNP              1
SWD         

#### Position

In [23]:
# Impute position
la_frame_up.POSITION=la_frame_up.POSITION.apply(lambda x: 1 if x==1 else 0)

In [24]:
la_frame_up.head()

,DIST_NAME,AC_NAME,AC_TYPE,CAND_NAME,CAND_SEX,CAND_CATEGORY,CAND_AGE,PARTYABBRE,TOTALVALIDVOTESPOLLED,POSITION
1572,saharanpur,behat,GEN,mahaveersinghrana,M,GEN,50,BSP,70274,1
1573,saharanpur,behat,GEN,naresh,M,GEN,47,INC,69760,0
1574,saharanpur,behat,GEN,umaralikhan,M,GEN,34,SP,47366,0
1575,saharanpur,behat,GEN,ajaychauhan,M,GEN,29,BJP,23623,0
1576,saharanpur,behat,GEN,dharamsingh,M,SC,39,CPI,2251,0


In [25]:
la_frame_up['incumb_match_key']=la_frame_up.AC_NAME.map(str)+la_frame_up.CAND_NAME.map(str)+la_frame_up.PARTYABBRE.map(str)+la_frame_up.POSITION.map(str)

In [33]:
la_frame_up.head()
print(la_frame_up.shape)

(7031, 11)


In [35]:
la_frame_up=la_frame_up[la_frame_up['POSITION']==1]

In [36]:
la_frame_up.shape

(403, 11)

In [37]:
# save the result dataframe to excel csv file
la_frame_up.to_csv('incumb_2012.csv')